Installing the required libraries

In [30]:
!pip install langchain
!pip install langchain-community
!pip install wikipedia
!pip install Chromadb


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.29.3 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.



  Obtaining dependency information for Chromadb from https://files.pythonhosted.org/packages/28/8e/5c186c77bf749b6fe0528385e507e463f1667543328d76fd00a49e1a4e6a/chromadb-0.6.3-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/84/c2/80633736cd183ee4a62107413def345f7e6e3c01563dbca1417363cf957e/build-1.2.2.post1-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.6 from https://files.pythonhosted.org/packages/3e/79/1bce519cf186112d6d5ce2985392a89528c6e1e9332d680bf752694a4cdf/chroma_hnswlib-0.7.6-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/52/b3/7e4df40e585df024fac2f80d1a2d579c854ac37109675db2b0cc22c0bb9e/fastapi-0.115.6-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/61/14/33a3a1352cfa71812a3a21e8c9bfb83f60b0

Using the WikipediaLoader provided by LangChain an article related to cat has been fetched and stored in docs, so that this document can be stored in Chroma database and referred to as the source.
Only one document has been loaded by using load_max_docs=1 parameter and the query passed is "cats" which enables to get some information about cats from Wikipedia

In [19]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = WikipediaLoader(
    query="cats",  # Wikipedia topic
    lang="en",                        # Language of the article
    load_max_docs=1                  # Fetch only 1 documen
    
)

# Load content
docs = loader.load()
print(docs)

[Document(metadata={'title': 'Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, purring, trilling, hissing, growling, and grunting—as well as body language. It can hear sounds too faint or too high in frequency for human

Splitting is done and it is required, as the chunks of splitted data is better in memory management, token size limitations, improved search accuracy.

In [37]:
# Initialize the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", ".", " "],
    #add_start_index=True
)

# Split the text
data = text_splitter.split_documents(docs)
for chunk in data:
    print('\n\n',chunk,'\n\n')




 page_content='The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous' metadata={'title': 'Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, purring, trilling, hissing, growlin

In [32]:
!pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/1e/86/eea2309dc258fb86c7d9b10db536434fc16420feaa3b6113df18b23db7c2/tiktoken-0.8.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 10.2/884.5 kB ? eta -:--:--
   - ------------------------------------- 30.7/884.5 kB 435.7 kB/s eta 0:00:02
   ---- --------------------------------- 112.6/884.5 kB 930.9 kB/s eta 0:00:01
   --------- ------------------------------ 204.8/884.5 kB 1.2 MB/s eta 0:00:01
   ------------------ --------------------- 409.6/884.5 kB 2.1 MB/s eta 0:00:01
   ------------------------ --------------- 532.5/884.5 kB 2.4 MB/s eta 0:00:01
   ------------------------ --------------- 532.5/884.5 kB 2.4 MB/s eta 0:00:01
   ------------------------ --------------- 532.5/884.5 kB 2.4 MB/s eta 0:00:01
   ------------------------ --------------- 532.5/884.5 kB 2.4 MB/s eta 0:00:01
   ----

OpenAIEmbeddings class has been used to generate vector embeddings from text data and Chroma.from_documents method has been used to store these embeddings to Chroma database.
Custom IDs has also been given, rather than using default Ids.

In [38]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()
store = Chroma.from_documents(
    data,
    embeddings, 
    ids = [f"{item.metadata['source']}-{index}" for index,item in enumerate(data)],
    collection_name = "cats-embeddings",
    persist_directory='db'
)
store.persist()

Sample of how an index looks like.

In [55]:
#data

for index,item in enumerate(data):
    print(index)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


PromptTemplate has been used with context and question variables. They are used for,

context: Contains the information retrieved from a vector store (e.g., Chroma) based on the query.
question: Specifies the user’s input query.

RetrievalQA has been used so that it combines a retrieval mechanism (from our Chroma db) with a language model, here it is gpt-4o-mini to process and generate answers based on the retrieved information. 

return_source_documents has been set to true, so that the chain returns the documents retrieved from the retriever alongside the answer.

System message has also been passed, which instructs the way we want "You are a helpful bot that would answer only about cats.  If anythig else is asked just say, I do not know."

return_source_documents has been set to True, so that it can give the source information.

In [47]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pprint

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="You are a helpful bot that would answer only about cats.  If anythig else is asked just say, I do not know.  \n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

qa_with_source = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=store.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True,
)



For the question, "Does a cat exist", it was able to answer correctly saying yes they do

In [48]:
pprint.pprint(qa_with_source("Does a cat exist"))

C:\Users\Riaz\AppData\Local\Temp\ipykernel_53380\764451507.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  pprint.pprint(qa_with_source("Does a cat exist"))


{'query': 'Does a cat exist',
 'result': 'Yes, a cat exists. Specifically, domestic cats, known '
           'scientifically as Felis catus, are a distinct species.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a cre

For the below question, "Can my professor drive a car", it just says I do not know.  
Because, that was the instruction given by us (You are a helpful bot that would answer only about cats.  If anythig else is asked just say, I do not know.) in the initial system message.

In [49]:
pprint.pprint(qa_with_source("Can my professor drive a car"))

{'query': 'Can my professor drive a car',
 'result': 'I do not know.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, purring, trilling

For the below question, "Are ginger cats more liked", it just says I do not know.  
I am not sure, why it says like that as this topic is still related to cats.  However, this specific topic is not mentioned in RAG.  But, why is it not able to get the result from general model outside of RAG.

In [50]:
pprint.pprint(qa_with_source("Are ginger cats more liked"))

{'query': 'Are ginger cats more liked',
 'result': 'I do not know.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, purring, trilling, 

For the below question, "Did the domestication of cat happened in USA", it just says I do not know.  
I am not sure, why it says like that as this topic is still related to cats.  However, this specific topic is not mentioned in RAG.  But, why is it not able to get the result from general model outside of RAG.

In [51]:
pprint.pprint(qa_with_source("Did the domestication of cat happened in USA"))

{'query': 'Did the domestication of cat happened in USA',
 'result': 'I do not know.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations—including meowing, p

Correct response has been given for the question, "Where did the first domestication of cat happened"

In [53]:
pprint.pprint(qa_with_source("Where did the first domestication of cat happened"))

{'query': 'Where did the first domestication of cat happened',
 'result': 'The first domestication of cats happened in the Near East around '
           '7500 BC.',
 'source_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Cat', 'summary': 'The cat (Felis catus), also referred to as the domestic cat, is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepu